# MSV / SS 2023 - Übung 3 - Lösung der Hausaufgaben

## Hausaufgaben

### Übung 3.1
Implementieren Sie eine binäre Version des Klassifiers (Daten aus Jurafsky & Martin)

In [11]:
import pandas as pd

#### Training Set

In [4]:
sentiment_train = [('-', 'just plain boring'),
              ('-', 'entirely predictable and lacks energy'),
             ('-', 'no surprises and very few laughs'),
             ('+', 'very powerful'),
             ('+', 'the most fun film of the summer')]  

sentiment_train = pd.DataFrame(sentiment_train, columns = ['category' , 'document'])

print(sentiment_train)

  category                               document
0        -                      just plain boring
1        -  entirely predictable and lacks energy
2        -       no surprises and very few laughs
3        +                          very powerful
4        +        the most fun film of the summer


#### Wortschatz

In [16]:
vocabulary = []

for i in range(len(sentiment_train)):
    
    for w in sentiment_train.loc[i, "document"].split():
        
        if w not in vocabulary: vocabulary.append(w)
            
print("Wortschatzgrösse: "+str(len(vocabulary)))

Wortschatzgrösse: 20


#### Prior - P(c)

In [13]:
P_pos = len(sentiment_train[sentiment_train.category == "+"]) / len(sentiment_train)
P_pos

0.4

In [14]:
P_neg = len(sentiment_train[sentiment_train.category == "-"]) / len(sentiment_train)
P_neg

0.6

#### Likelihood - P(d|c)

#### Dokumentensammlungen für die 2 Klassen (Bag of Words)

In [18]:
dok_pos = {}
dok_neg = {}


for i in range(len(sentiment_train)):
    
    if sentiment_train.loc[i, "category"] == "+": 
        # Ein Set darf keine Duplikate enthalten
        for w in set(sentiment_train.loc[i, "document"].split()):
            if w in dok_pos.keys(): dok_pos[w] += 1
            else: dok_pos[w] = 1
    else: 
        for w in set(sentiment_train.loc[i, "document"].split()):
            if w in dok_neg.keys(): dok_neg[w] += 1
            else: dok_neg[w] = 1
                
print("+:", dok_pos)
print("-:", dok_neg)

+: {'very': 1, 'powerful': 1, 'film': 1, 'most': 1, 'fun': 1, 'summer': 1, 'of': 1, 'the': 1}
-: {'just': 1, 'boring': 1, 'plain': 1, 'energy': 1, 'lacks': 1, 'predictable': 1, 'entirely': 1, 'and': 2, 'laughs': 1, 'few': 1, 'very': 1, 'surprises': 1, 'no': 1}


#### Features: P(w|c) Schätzung (mit Add-1-Smoothing)

In [19]:
F_w_pos = {}
F_w_neg = {}

P_w_pos = {}
P_w_neg = {}

for w in vocabulary:
    
    F_w_pos[w] = 1 
    F_w_neg[w] = 1
    
    if w in dok_pos.keys():
        F_w_pos[w] += dok_pos[w]
        

    if w in dok_neg.keys():
        F_w_neg[w] += dok_neg[w]
        
    P_w_pos[w] = F_w_pos[w]/(sum(dok_pos.values())+len(vocabulary))
    P_w_neg[w] = F_w_neg[w]/(sum(dok_neg.values())+len(vocabulary))
    
print("+:", P_w_pos)
print("-:", P_w_neg)

+: {'just': 0.03571428571428571, 'plain': 0.03571428571428571, 'boring': 0.03571428571428571, 'entirely': 0.03571428571428571, 'predictable': 0.03571428571428571, 'and': 0.03571428571428571, 'lacks': 0.03571428571428571, 'energy': 0.03571428571428571, 'no': 0.03571428571428571, 'surprises': 0.03571428571428571, 'very': 0.07142857142857142, 'few': 0.03571428571428571, 'laughs': 0.03571428571428571, 'powerful': 0.07142857142857142, 'the': 0.07142857142857142, 'most': 0.07142857142857142, 'fun': 0.07142857142857142, 'film': 0.07142857142857142, 'of': 0.07142857142857142, 'summer': 0.07142857142857142}
-: {'just': 0.058823529411764705, 'plain': 0.058823529411764705, 'boring': 0.058823529411764705, 'entirely': 0.058823529411764705, 'predictable': 0.058823529411764705, 'and': 0.08823529411764706, 'lacks': 0.058823529411764705, 'energy': 0.058823529411764705, 'no': 0.058823529411764705, 'surprises': 0.058823529411764705, 'very': 0.058823529411764705, 'few': 0.058823529411764705, 'laughs': 0.0

### Testing

In [20]:
sentiment_test = 'predictable with no fun'

In [21]:
P_test_pos = P_pos
P_test_neg = P_neg
    
for w in sentiment_test.split():
        
    if w in vocabulary:    
        
        P_test_pos = P_test_pos*P_w_pos[w]
        P_test_neg = P_test_neg*P_w_neg[w]
            
print("P positive for test: ",P_test_pos)
print("P negative for test: ",P_test_neg)

P positive for test:  3.644314868804664e-05
P negative for test:  6.106248727864848e-05


Die vorhergesagte Klasse für den Test ist 'negativ'